In [1]:
from pandas import DataFrame, Series
import pandas as pd
import numpy as np
from IPython.display import display

# DataFrame
## 创建
和Series一样,DataFrame也可以接收多种数据类型的数据。

### dict of dict/Series
这时DataFrame会用dict的keys作为DataFrame的columns,内层Series的index的并集(或者内层dict的keys的并集)作为DataFrame的index。 内层的Series和Dict长度可以不一样。

In [2]:
d = {
    'one' : Series([1., 2., 3.], index=['a', 'b', 'c']),
    'two' : Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])
}

df = DataFrame(d)

display(df)

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


In [3]:
d = {
    'one' : {'a':1,'b':2,'c':3},
    'two' : {'a':1,'b':2,'c':3,'d':4}
}

df = DataFrame(d)

display(df)

,one,two
a,1.0,1
b,2.0,2
c,3.0,3
d,NaN,4


如果额外的传入index,columns参数。那么传入的数据中不在传入index,columns范围内的会被舍弃，传入index,columns中需要，但传入数据中找不到对应值的部分会被用NaN填充。

In [4]:
d = {
    'one' : {'a':1,'b':2,'c':3},
    'two' : {'a':1,'b':2,'c':3,'d':4}
}

df = DataFrame(d,index = ['a','b','e'],columns = ['one','two','three'])

display(df)

,one,two,three
a,1.0,1.0,NaN
b,2.0,2.0,NaN
e,NaN,NaN,NaN


如果希望把外层dict的keys作为index，内层Series的index并集(或者内层dict的keys的并集)作为columns的话，可以用DataFrame.from_dict实现。

In [5]:
d = {
    'one' : {'a':1,'b':2,'c':3},
    'two' : {'a':1,'b':2,'c':3,'d':4}
}

DataFrame.from_dict(d,orient='index')

,c,b,d,a
one,3,2,NaN,1
two,3,2,4.0,1


注意要把orient设置成'index'(默认是'columns'),否则就和```DataFrame(d)```的效果一样了。

In [6]:
d = {
    'one' : {'a':1,'b':2,'c':3},
    'two' : {'a':1,'b':2,'c':3,'d':4}
}

DataFrame.from_dict(d)

,one,two
a,1.0,1
b,2.0,2
c,3.0,3
d,NaN,4


### dict of list/ndarray

dict内层也可以是list或者ndarray。这时由于没有可利用的index对成员进行匹配，内层list或者ndarry的长度必须一致。

In [7]:
d = {
    'one' : [1., 2., 3., 4.],
    'two' : [4., 3., 2., 1.]
}

DataFrame.from_dict(d)

,one,two
0,1.0,4.0
1,2.0,3.0
2,3.0,2.0
3,4.0,1.0


In [8]:
d = {
    'one' : np.array([1., 2., 3., 4.]),
    'two' : np.array([4., 3., 2., 1.])
}

DataFrame.from_dict(d)

,one,two
0,1.0,4.0
1,2.0,3.0
2,3.0,2.0
3,4.0,1.0


也支持DataFrame.from_dict方法

In [9]:
d = {
    'one' : {'a':1,'b':2,'c':3},
    'two' : {'a':1,'b':2,'c':3,'d':4}
}

DataFrame.from_dict(d,orient='index')

,c,b,d,a
one,3,2,NaN,1
two,3,2,4.0,1


### list of dict

数据可以是list of dict。这时会将内层dict的keys作为组成DataFrame的columns。内侧dict的长度可以不一致。

In [10]:
d = [{'a': 1, 'b': 2}, {'a': 5, 'b': 10, 'c': 20}]

DataFrame(d)

,a,b,c
0,1,2,NaN
1,5,10,20.0


In [11]:
d = [{'a': 1, 'b': 2}, {'a': 5, 'b': 10, 'c': 20}]

DataFrame.from_dict(d)

,a,b,c
0,1,2,NaN
1,5,10,20.0


### list of tuple
这时会将list中的tuple作为每一个行的

In [12]:
d = [
    (1,2,),
    (3,4),
    (5,6),    
]

DataFrame(d)

,0,1
0,1,2
1,3,4
2,5,6


也可以在内层的tuple中指定label并传入DataFrame.from_items。 相比于dict方式,这种方式可以确保生成的columns顺序是和list中出现的顺序一致的。

In [13]:
d = [
    ('A',(1,2)),
    ('B',(3,4)),
    ('C',(5,6)),
]

DataFrame.from_items(d)

,A,B,C
0,1,3,5
1,2,4,6


## 数据的选择

为了方面后面代码实验，先定义一个函数生成我们需要的测试DataFrame

In [14]:
def get_df():
    d = {
        'one' : Series([1., 2., 3.], index=['a', 'b', 'c']),
        'two' : Series([1., 2., 3., 4.], index=['a', 'b', 'c', 'd'])
    }

    df = DataFrame(d)
    
    return df

df = get_df()

### 选择单列对应的Series
#### 根据label选择

传入代表列的label即可,得到的结果是一个Series。

In [94]:
df['one']

a    1.0
b    2.0
c    3.0
d    NaN
Name: one, dtype: float64

#### 根据位置选择
如果要选择第n列，可以使用```iloc[:,n-1]```,例如我们要选择第二列的话。

In [95]:
df.iloc[:,1]

a    1.0
b    2.0
c    3.0
d    4.0
Name: two, dtype: float64

### 选择多列对应的DataFrame

#### 根据label进行选择

将所需的label组成list传入[]即可。

In [98]:
display(df[['one','two']])

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


注意即使list中只传入一个label，返回的结果依然是DataFrame。

In [99]:
display(df[['one']])

,one
a,1.0
b,2.0
c,3.0
d,NaN


#### 根据位置进行选择
在iloc中选择

In [102]:
display(df.iloc[:,[0,1]])

,one,two
a,1.0,1.0
b,2.0,2.0
c,3.0,3.0
d,NaN,4.0


## 修改列

## 新增列

In [105]:
df = get_df()

df['four'] = df['one']+df['two']

display(df)

,one,two,four
a,1.0,1.0,2.0
b,2.0,2.0,4.0
c,3.0,3.0,6.0
d,NaN,4.0,NaN


In [106]:
df = get_df()

df['four'] = (df['one']+df['two']).to_frame()

display(df)

,one,two,four
a,1.0,1.0,2.0
b,2.0,2.0,4.0
c,3.0,3.0,6.0
d,NaN,4.0,NaN


如果有inndex,可以行数不一致。

In [109]:
df = get_df()

df['four'] = (df['one']+df['two']).iloc[0:2]

display(df)

,one,two,four
a,1.0,1.0,2.0
b,2.0,2.0,4.0
c,3.0,3.0,NaN
d,NaN,4.0,NaN


如果没有index，则需要行数一致。

In [113]:
df = get_df()

df['four'] = [1,2,3,4]
display(df)

,one,two,four
a,1.0,1.0,1
b,2.0,2.0,2
c,3.0,3.0,3
d,NaN,4.0,4


增加多列

右侧的column不需要和左侧一致。

In [122]:
df = get_df()

df[['four','five']] = df[['one','two']]
display(df)

,one,two,four,five
a,1.0,1.0,1.0,1.0
b,2.0,2.0,2.0,2.0
c,3.0,3.0,3.0,3.0
d,NaN,4.0,NaN,4.0


右侧如果是ndarray,则不能用于创建新列。

In [124]:
df = get_df()

try:
    df[['four','five']] = df[['one','two']].values
except Exception as e:
    print(type(e),e)

<class 'KeyError'> "['four' 'five'] not in index"
